In [5]:
from sympy import *
import numpy as np

In [6]:
"""
author : YorozuyaSaint
https://github.com/berylgithub
"""

def forward_diff_parabolic(u, range_x, range_t, h, k, alpha, log=True):
    """
    numerical solution using forward difference method for parabolic heat equation
    """

    x=Symbol('x')
    t=Symbol('t')
    if log:
        print("u(x,t) = ",u(x,t))
        print("h=",h, ";x=",range_x[1])
        print("k=",k, ";t=",range_t[1])
        print("alpha=",alpha)
        
    l = (alpha**2)*k/(h**2)
    max_iter_i = int((range_x[1]-range_x[0])/h) + 1
    max_iter_j = int((range_t[1]-range_t[0])/k) + 1
    m = max_iter_i-1
    #initiate tridiagonal matrix A 
    A = np.zeros((m-1, m-1))
    for i in range(m-1):
        for j in range(m-1):
            if i==0:
                A[i] = np.array( [1-(2*l), l]+([0]*(len(A)-2)) )
            elif i==len(A)-1:
                A[i] = np.array( ([0]*(len(A)-2))+[l, 1-(2*l)] )
            else:
                A[i][i-1] = l
                A[i][i] = (1-(2*l))
                A[i][i+1] = l
    #insert zeros for u(0,t) and u(l,t)
    #A=np.insert(A, 0, np.zeros(m-1), 0)
    #A=np.insert(A, len(A), np.zeros(m-1), 0)
    if log:
        print("A = ")
        print(A)

    #initiate vector x input
    xi=np.array([h*i for i in range(max_iter_i)])

    #initiate w with size (m+1) * (m+1) (2 rows for u(0,t) and u(l,t))
    #w = np.zeros((max_iter_i, max_iter_j))
    w = np.zeros((m-1, max_iter_j))
    
    #initiate vector w at j=0, where 1<=i<=m-1, where w(i,0) = f(xi) 
    w10 = np.array([u(x,t).evalf(subs={x:xi[i], t:0}) for i in range(1, m)])
    
    #add zeros at both 0 and l
    #w10=np.insert(w10, 0, 0, 0)
    #w10=np.insert(w10, len(w10), 0, 0)
    
    #set it as w(1), but transpose w first, then insert w(1,0)
    w = np.transpose(w)
    w[0] = w10
    #w = np.transpose(w)
    #print(np.transpose(w))
    
    #calculate the w at j>0 until j*k = t
    for j in range(1, len(w)):
        w[j] = np.matmul(A, np.transpose(w[j-1]))
    if log :
        print("w(",len(w)-1,") = ")
        print(w[-1])
    return w[-1]


def backward_diff_parabolic(u, l, T, m, N, alpha, log=True):
    """
    backward difference solution to parabolic heat partial differential equation
    """
    x=Symbol('x')
    t=Symbol('t')
    if log:
        print("u(x,t) = ",u(x,t))
        print("l = ",l)
        print("T = ",T)
        print("m = ",m)
        print("N = ",N)
        print("alpha = ",alpha)
        
    #step 1
    h=l/m
    k=T/N
    var_lambda = (alpha**2)*k/(h**2)
    
    #step 2
    w = np.zeros(m)
    for i in range(1, m):
        w[i] = u(x,t).evalf(subs={x:(i*h), t:0})
    
    #step 3
    l = np.zeros(m)
    u = np.zeros(m)
    l[1] = 1+(2*var_lambda)
    u[1] = -var_lambda/l[1]
    
    #step 4
    for i in range(2, m-1):
        l[i] = 1 + (2*var_lambda) + (var_lambda*(u[i-1]))
        u[i] = -var_lambda/l[i]
        
    #step 5
    l[m-1] = 1 + (2*var_lambda) + (var_lambda*(u[m-2]))
    
    #step 6
    if log :
        print("Backward diff :")
    z = np.zeros(m)
    for j in range(1, N+1):
        #step 7
        t = j*k
        z[1] = w[1]/l[1]
        #step 8
        for i in range(2, m):
            z[i] = (w[i] + var_lambda*z[i-1])/l[i]
        #step 9
        w[m-1] = z[m-1]
        #step 10
        for i in range(m-2, 0, -1):
            w[i] = z[i] - (u[i]*w[i+1])
        #step 11
        if log:
            print("j iter = ",j)
            print("x\tw[i]")
        for i in range(1, m):
            x = i*h
            
            if log:
                print(x,"\t",w[i])
        if log:
            print()
    
    return w

def crank_nicolson_parabolic(u, l, T, m, N, alpha, log=True):
    """
    solves parabolic partial differential equation using crank-nicolson algorithm
    """
    x=Symbol('x')
    t=Symbol('t')
    
    if log:
        print("u(x,t) = ",u(x,t))
        print("l = ",l)
        print("T = ",T)
        print("m = ",m)
        print("N = ",N)
        print("alpha = ",alpha)
        
    #step 1
    h=l/m
    k=T/N
    var_lambda = (alpha**2)*k/(h**2)
    w = np.zeros(m+1)
    
    #step 2
    for i in range(1, m):
        w[i] = u(x,t).evalf(subs={x:(i*h), t:0})
    
    #step 3
    l = np.zeros(m)
    u = np.zeros(m)
    l[1] = 1+var_lambda
    u[1] = -var_lambda/(2*l[1])
    
    #step 4
    for i in range(2, m-1):
        l[i] = 1 + var_lambda + (var_lambda*(u[i-1])/2)
        u[i] = -var_lambda/(2*l[i])
        
    #step 5
    l[m-1] = 1 + var_lambda + (var_lambda*(u[m-2])/2)
    
    #step 6
    if log :
        print("Crank-Nicolson Algorithm :")
    z = np.zeros(m)
    for j in range(1, N+1):
        #step 7
        t = j*k
        z[1] = ( ((1-var_lambda)*w[1]) + (var_lambda*w[2]/2) )/l[1]
        #step 8
        for i in range(2, m):
            z[i] = (((1-var_lambda)*w[i]) + (var_lambda*(w[i+1] + w[i-1] + z[i-1])/2) )/l[i]
        #step 9
        w[m-1] = z[m-1]
        #step 10
        for i in range(m-2, 0, -1):
            w[i] = z[i] - (u[i]*w[i+1])
        #step 11
        if log:
            print("j iter = ",j)
            print("x\tw[i]")
        for i in range(1, m):
            x = i*h
            
            if log:
                print(x,"\t",w[i])
        if log:
            print()
    
    return w


In [235]:
#Example 1 page 727
print("Example 1 page 727")
u = lambda x, t : exp((-1*(pi)**2)*t)*sin(pi*x)

#variables initialization
alpha = 1
k = 0.0005
h = 0.1
rn_x = [0,1]
rn_t = [0,0.5]
x_iter = int((rn_x[1]-rn_x[0])/h)
xi = np.array([h*i for i in range(1, x_iter)])

#exact calculation of u(x,t)
u_exact = np.array([u(x,t).evalf(subs={x:xi[i], t:rn_t[1]}) for i in range(len(xi))])

#approximation of u(x,t)
w_last = forward_diff_parabolic(u, rn_x, rn_t, h, k, alpha)

print("\nResult :")
print("x\tu(x,0.5)\tw(i,1000)\terror")
for i in range(len(w_last)):
    err = abs(u_exact[i] - w_last[i])
    print(xi[i],"\t",u_exact[i],"\t",w_last[i],"\t",err)

Example 1 page 727
u(x,t) =  exp(-pi**2*t)*sin(pi*x)
h= 0.1 ;x= 1
k= 0.0005 ;t= 0.5
alpha= 1
A = 
[[0.9  0.05 0.   0.   0.   0.   0.   0.   0.  ]
 [0.05 0.9  0.05 0.   0.   0.   0.   0.   0.  ]
 [0.   0.05 0.9  0.05 0.   0.   0.   0.   0.  ]
 [0.   0.   0.05 0.9  0.05 0.   0.   0.   0.  ]
 [0.   0.   0.   0.05 0.9  0.05 0.   0.   0.  ]
 [0.   0.   0.   0.   0.05 0.9  0.05 0.   0.  ]
 [0.   0.   0.   0.   0.   0.05 0.9  0.05 0.  ]
 [0.   0.   0.   0.   0.   0.   0.05 0.9  0.05]
 [0.   0.   0.   0.   0.   0.   0.   0.05 0.9 ]]
w( 1000 ) = 
[0.00228652 0.00434922 0.00598619 0.00703719 0.00739934 0.00703719
 0.00598619 0.00434922 0.00228652]

Result :
x	u(x,0.5)	w(i,1000)	error
0.1 	 0.00222241417851267 	 0.0022865207865784528 	 6.41066080657787e-5
0.2 	 0.00422728297276244 	 0.004349220987439515 	 0.000121938014677076
0.30000000000000004 	 0.00581835585642586 	 0.005986189135245531 	 0.000167833278819674
0.4 	 0.00683988752999332 	 0.007037187382261514 	 0.000197299852268192
0.5 	 0.00719

In [44]:
#Example 2 page 731
print("Example 1 page 731")
u = lambda x, t : exp((-1*(pi)**2)*t)*sin(pi*x)

#variables initialization
l = 1
T = 0.5
m = 10
N = 50
alpha = 1
x_iter = int(m)
xi = np.array([h*i for i in range(1, x_iter)])

#exact calculation of u(x,t)
u_exact = np.array([u(x,t).evalf(subs={x:xi[i], t:T}) for i in range(len(xi))])

#approximation of u(x,t) using backward difference
w_last = backward_diff_parabolic(u, l, T, m, N, alpha, log=False)
w_last = w_last[1:]

print("\nResult :")
print("x\tu(x,0.5)\tw(i,50)\terror")
for i in range(len(w_last)):
    err = abs(u_exact[i] - w_last[i])
    print(xi[i],"\t",u_exact[i],"\t",w_last[i],"\t",err)

Example 1 page 731

Result :
x	u(x,0.5)	w(i,50)	error
0.314159265358979 	 0.00600057911579518 	 0.0028980166450535885 	 0.00310256247074159
0.628318530717959 	 0.00661538012684316 	 0.005512355229220069 	 0.00110302489762309
0.942477796076938 	 0.00129259265614838 	 0.0075871060767132344 	 0.00629451342056486
1.25663706143592 	 -0.00519035237459191 	 0.008919178118941287 	 0.0141095304935332
1.57079632679490 	 -0.00701473268423438 	 0.009378178863319288 	 0.0163929115475537
1.88495559215388 	 -0.00254308839819121 	 0.008919178118941284 	 0.0114622665171325
2.19911485751286 	 0.00421108721509700 	 0.0075871060767132275 	 0.00337601886161622
2.51327412287183 	 0.00718563074911635 	 0.005512355229220063 	 0.00167327551989628
2.82743338823081 	 0.00371076131639063 	 0.0028980166450535854 	 0.000812744671337047


In [46]:
#Example 3 page 735
print("Example 3 page 735")
u = lambda x, t : exp((-1*(pi)**2)*t)*sin(pi*x)

#variables initialization
l = 1
T = 0.5
m = 10
N = 50
alpha = 1
x_iter = int(m)
xi = np.array([h*i for i in range(1, x_iter)])

#exact calculation of u(x,t)
u_exact = np.array([u(x,t).evalf(subs={x:xi[i], t:T}) for i in range(len(xi))])

#approximation of u(x,t) using backward difference
w_last = crank_nicolson_parabolic(u, l, T, m, N, alpha, log=False)
w_last = w_last[1:-1]

print("\nResult :")
print("x\tu(x,0.5)\tw(i,50)\terror")
for i in range(len(w_last)):
    err = abs(u_exact[i] - w_last[i])
    print(xi[i],"\t",u_exact[i],"\t",w_last[i],"\t",err)

Example 3 page 735

Result :
x	u(x,0.5)	w(i,50)	error
0.314159265358979 	 0.00600057911579518 	 0.0023051233677887873 	 0.00369545574800639
0.628318530717959 	 0.00661538012684316 	 0.004384605199599512 	 0.00223077492724365
0.942477796076938 	 0.00129259265614838 	 0.0060348913251326705 	 0.00474229866898429
1.25663706143592 	 -0.00519035237459191 	 0.007094440240201527 	 0.0122847926147934
1.57079632679490 	 -0.00701473268423438 	 0.007459535914687764 	 0.0144742685989221
1.88495559215388 	 -0.00254308839819121 	 0.007094440240201526 	 0.00963752863839274
2.19911485751286 	 0.00421108721509700 	 0.006034891325132668 	 0.00182380411003566
2.51327412287183 	 0.00718563074911635 	 0.00438460519959951 	 0.00280102554951684
2.82743338823081 	 0.00371076131639063 	 0.0023051233677887865 	 0.00140563794860185


## Exercise 12.2 No. 5,7,9

## FTCS

### No. 5.a

In [24]:
#Exercise 12.2, No.5,7,9
u = lambda x,t: exp(-4*(pi**2)*t)*sin(2*pi*x)
x = Symbol('x')
t = Symbol('t')

#init variable
#No.5a part 1
print("Exercise 5.a - part 1")
rn_x = [0,2]
rn_t = [0,0.5]
h = 0.4
k = 0.1
alpha = 1
x_iter = int((rn_x[1]-rn_x[0])/h)
xi = np.array([h*i for i in range(1, x_iter)])
#exact calculation of u(x,t)
u_exact = np.array([u(x,t).evalf(subs={x:xi[i], t:rn_t[1]}) for i in range(len(xi))])
#approximation of u(x,t)
w_last_1 = forward_diff_parabolic(u, rn_x, rn_t, h, k, alpha)

#No.5a part 2
print("\nExercise 5.a - part 2")
h = 0.4
k = 0.05
x_iter = int((rn_x[1]-rn_x[0])/h)
xi = np.array([h*i for i in range(1, x_iter)])
#approximation of u(x,t)
w_last_2 = forward_diff_parabolic(u, rn_x, rn_t, h, k, alpha)


print(u_exact)

print("\nResult :")
print("x\tu(x,0.5)\tw(i,5)\t|u(x,0.5)-w(i,5)|\tw(i,10)\t|u(x,0.5)-w(i,10)|")
for i in range(len(w_last)):
    print(format(xi[i],'.1f'),"\t",
          format(u_exact[i], '.12f'),"\t",
          format(w_last_1[i], '.5f'),"\t",
          format(abs(u_exact[i] - w_last_1[i]), '.5f'),"\t",
          format(w_last_2[i], '.12f'),"\t",
          format(abs(u_exact[i] - w_last_2[i]), '.12f') )
    


Exercise 5.a - part 1
u(x,t) =  exp(-4*pi**2*t)*sin(2*pi*x)
h= 0.4 ;x= 2
k= 0.1 ;t= 0.5
alpha= 1
A = 
[[-0.25   0.625  0.     0.   ]
 [ 0.625 -0.25   0.625  0.   ]
 [ 0.     0.625 -0.25   0.625]
 [ 0.     0.     0.625 -0.25 ]]
w( 5 ) = 
[-1.87612234  3.03562971 -3.03562971  1.87612234]

Exercise 5.a - part 2
u(x,t) =  exp(-4*pi**2*t)*sin(2*pi*x)
h= 0.4 ;x= 2
k= 0.05 ;t= 0.5
alpha= 1
A = 
[[0.375  0.3125 0.     0.    ]
 [0.3125 0.375  0.3125 0.    ]
 [0.     0.3125 0.375  0.3125]
 [0.     0.     0.3125 0.375 ]]
w( 10 ) = 
[ 8.50814426e-10 -1.37664664e-09  1.37664666e-09 -8.50814417e-10]
[1.57249482678860e-9 -2.54435007687733e-9 2.54435007687733e-9
 -1.57249482678860e-9]

Result :
x	u(x,0.5)	w(i,5)	|u(x,0.5)-w(i,5)|	w(i,10)	|u(x,0.5)-w(i,10)|
0.4 	 0.000000001572 	 -1.87612 	 1.87612 	 0.000000000851 	 0.000000000722
0.8 	 -0.000000002544 	 3.03563 	 3.03563 	 -0.000000001377 	 0.000000001168
1.2 	 0.000000002544 	 -3.03563 	 3.03563 	 0.000000001377 	 0.000000001168
1.6 	 -0.00000000157

### No. 5.b

In [29]:
#Exercise 12.2, No.5,7,9
u = lambda x,t: exp(-t)*sin(x)
x = Symbol('x')
t = Symbol('t')

#init variable
#No.5a part 1
print("Exercise 5.b - part 1")
rn_x = [0,pi.evalf()]
rn_t = [0,0.5]
h = pi.evalf()/10
k = 0.05
alpha = 1
x_iter = int((rn_x[1]-rn_x[0])/h)
xi = np.array([h*i for i in range(1, x_iter)])
#exact calculation of u(x,t)
u_exact = np.array([u(x,t).evalf(subs={x:xi[i], t:rn_t[1]}) for i in range(len(xi))])
#approximation of u(x,t)
w_last = forward_diff_parabolic(u, rn_x, rn_t, h, k, alpha)

print(xi)
print("\nResult :")
print("x\tu(x,0.5)\tw(i,10)\t|u(x,0.5)-w(i,10)|")
for i in range(len(w_last)):
    print(format(xi[i],'.2f'),"\t",
          format(u_exact[i], '.12f'),"\t",
          format(w_last[i], '.5f'),"\t",
          format(abs(u_exact[i] - w_last[i]), '.5f') )
    


Exercise 5.b - part 1
u(x,t) =  exp(-t)*sin(x)
h= 0.314159265358979 ;x= 3.14159265358979
k= 0.05 ;t= 0.5
alpha= 1
A = 
[[-0.01321184  0.50660592  0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.50660592 -0.01321184  0.50660592  0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.50660592 -0.01321184  0.50660592  0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.50660592 -0.01321184  0.50660592  0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.50660592 -0.01321184  0.50660592
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.50660592 -0.01321184
   0.50660592  0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.50660592
  -0.01321184  0.50660592  0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.50660592 -0.01321184  0.50660592]
 [ 0.          0.

## BTCS

### No 7.a

In [32]:
u = lambda x,t: exp(-4*(pi**2)*t)*sin(2*pi*x)
x = Symbol('x')
t = Symbol('t')

#init variable
#No.7a part 1
print("Exercise 7.a - part 1")
l = 2
T = 0.5
h = 0.4
k = 0.1
m = int(l/h)
N = int(T/k)
alpha = 1
x_iter = int(l/h)
xi = np.array([h*i for i in range(1, x_iter)])
#exact calculation of u(x,t)
u_exact = np.array([u(x,t).evalf(subs={x:xi[i], t:T}) for i in range(len(xi))])

#approximation of u(x,t) using backward difference
w_last_1 = backward_diff_parabolic(u, l, T, m, N, alpha)
w_last_1 = w_last_1[1:]

#No.5a part 2
print("\nExercise 7.a - part 2")
h = 0.4
k = 0.05
x_iter = int(l/h)
xi = np.array([h*i for i in range(1, x_iter)])
#approximation of u(x,t)
w_last_2 = backward_diff_parabolic(u, l, T, m, N, alpha)
w_last_2 = w_last_2[1:]



print("\nResult :")
print("x\tu(x,0.5)\tw(i,5)\t|u(x,0.5)-w(i,5)|\tw(i,10)\t|u(x,0.5)-w(i,10)|")
for i in range(len(w_last_1)):
    print(format(xi[i],'.1f'),"\t",
          format(u_exact[i], '.12f'),"\t",
          format(w_last_1[i], '.5f'),"\t",
          format(abs(u_exact[i] - w_last_1[i]), '.5f'),"\t",
          format(w_last_2[i], '.12f'),"\t",
          format(abs(u_exact[i] - w_last_2[i]), '.12f') )
    


Exercise 7.a - part 1
u(x,t) =  exp(-4*pi**2*t)*sin(2*pi*x)
l =  2
T =  0.5
m =  5
N =  5
alpha =  1
Backward diff :
j iter =  1
x	w[i]
0.4 	 0.18023194285349337
0.8 	 -0.2916214093953809
1.2000000000000002 	 0.29162140939538106
1.6 	 -0.18023194285349353

j iter =  2
x	w[i]
0.4 	 0.055264321617551536
0.8 	 -0.08941955074240389
1.2000000000000002 	 0.08941955074240394
1.6 	 -0.055264321617551584

j iter =  3
x	w[i]
0.4 	 0.016945637912425007
0.8 	 -0.027418618103352436
1.2000000000000002 	 0.027418618103352464
1.6 	 -0.01694563791242502

j iter =  4
x	w[i]
0.4 	 0.00519602224100798
0.8 	 -0.008407340592251284
1.2000000000000002 	 0.0084073405922513
1.6 	 -0.005196022241007982

j iter =  5
x	w[i]
0.4 	 0.0015932505620961882
0.8 	 -0.0025779335620664907
1.2000000000000002 	 0.002577933562066501
1.6 	 -0.001593250562096186


Exercise 7.a - part 2
u(x,t) =  exp(-4*pi**2*t)*sin(2*pi*x)
l =  2
T =  0.5
m =  5
N =  5
alpha =  1
Backward diff :
j iter =  1
x	w[i]
0.4 	 0.18023194285349337
0.8 

### No 7.b

In [37]:
u = lambda x,t: exp(-t)*sin(x)
x = Symbol('x')
t = Symbol('t')

#init variable
#No. 7.b
print("Exercise 7.b")
l = pi.evalf()
T = 0.5
h = pi.evalf()/10
k = 0.05
m = int(l/h)
N = int(T/k)
alpha = 1
x_iter = int(l/h)
xi = np.array([h*i for i in range(1, x_iter)])
#exact calculation of u(x,t)
u_exact = np.array([u(x,t).evalf(subs={x:xi[i], t:T}) for i in range(len(xi))])
#approximation of u(x,t)
w_last = backward_diff_parabolic(u, l, T, m, N, alpha)
w_last = w_last[1:]

print("\nResult :")
print("x\tu(x,0.5)\tw(i,10)\t|u(x,0.5)-w(i,10)|")
for i in range(len(w_last)):
    print(format(xi[i],'.2f'),"\t",
          format(u_exact[i], '.12f'),"\t",
          format(w_last[i], '.5f'),"\t",
          format(abs(u_exact[i] - w_last[i]), '.5f') )
    


Exercise 7.b
u(x,t) =  exp(-t)*sin(x)
l =  3.14159265358979
T =  0.5
m =  10
N =  10
alpha =  1
Backward diff :
j iter =  1
x	w[i]
0.314159265358979 	 0.2944168293586503
0.628318530717959 	 0.5600140881370054
0.942477796076938 	 0.7707932661209245
1.25663706143592 	 0.9061218287844539
1.57079632679490 	 0.9527528735245483
1.88495559215388 	 0.906121828784454
2.19911485751286 	 0.7707932661209246
2.51327412287183 	 0.5600140881370055
2.82743338823081 	 0.29441682935865043

j iter =  2
x	w[i]
0.314159265358979 	 0.28050648018544067
0.628318530717959 	 0.5335550316867614
0.942477796076938 	 0.7343754991900826
1.25663706143592 	 0.8633101761377072
1.57079632679490 	 0.9077380380092839
1.88495559215388 	 0.8633101761377073
2.19911485751286 	 0.7343754991900828
2.51327412287183 	 0.5335550316867617
2.82743338823081 	 0.2805064801854408

j iter =  3
x	w[i]
0.314159265358979 	 0.2672533550389354
0.628318530717959 	 0.5083460896230434
0.942477796076938 	 0.6996783670993757
1.25663706143592 	 0.

## Crank Nicolson

### No 9.a

In [41]:
u = lambda x,t: exp(-4*(pi**2)*t)*sin(2*pi*x)
x = Symbol('x')
t = Symbol('t')

#init variable
#No.9a part 1
print("Exercise 9.a - part 1")
l = 2
T = 0.5
h = 0.4
k = 0.1
m = int(l/h)
N = int(T/k)
alpha = 1
x_iter = int(l/h)
xi = np.array([h*i for i in range(1, x_iter)])
#exact calculation of u(x,t)
u_exact = np.array([u(x,t).evalf(subs={x:xi[i], t:T}) for i in range(len(xi))])

#approximation of u(x,t) using backward difference
w_last_1 = crank_nicolson_parabolic(u, l, T, m, N, alpha)
w_last_1 = w_last_1[1:-1]

#No.5a part 2
print("\nExercise 9.a - part 2")
h = 0.4
k = 0.05
x_iter = int(l/h)
xi = np.array([h*i for i in range(1, x_iter)])
#approximation of u(x,t)
w_last_2 = crank_nicolson_parabolic(u, l, T, m, N, alpha)
w_last_2 = w_last_2[1:-1]



print("\nResult :")
print("x\tu(x,0.5)\tw(i,5)\t|u(x,0.5)-w(i,5)|\tw(i,10)\t|u(x,0.5)-w(i,10)|")
for i in range(len(w_last_1)):
    print(format(xi[i],'.1f'),"\t",
          format(u_exact[i], '.12f'),"\t",
          format(w_last_1[i], '.7f'),"\t",
          format(abs(u_exact[i] - w_last_1[i]), '.7f'),"\t",
          format(w_last_2[i], '.12f'),"\t",
          format(abs(u_exact[i] - w_last_2[i]), '.12f') )
    


Exercise 9.a - part 1
u(x,t) =  exp(-4*pi**2*t)*sin(2*pi*x)
l =  2
T =  0.5
m =  5
N =  5
alpha =  1
Crank-Nicolson Algorithm :
j iter =  1
x	w[i]
0.4 	 -0.03603886603522751
0.8 	 0.05831211016100237
1.2000000000000002 	 -0.058312110161002295
1.6 	 0.03603886603522746

j iter =  2
x	w[i]
0.4 	 0.0022096503102783283
0.8 	 -0.003575289305282028
1.2000000000000002 	 0.0035752893052820567
1.6 	 -0.0022096503102783196

j iter =  3
x	w[i]
0.4 	 -0.0001354802476010191
0.8 	 0.0002192116454227337
1.2000000000000002 	 -0.0002192116454227141
1.6 	 0.00013548024760103036

j iter =  4
x	w[i]
0.4 	 8.306697853806942e-06
0.8 	 -1.3440519461714596e-05
1.2000000000000002 	 1.344051946172977e-05
1.6 	 -8.306697853797659e-06

j iter =  5
x	w[i]
0.4 	 -5.09308408092693e-07
0.8 	 8.240783150642584e-07
1.2000000000000002 	 -8.240783150523501e-07
1.6 	 5.093084081000433e-07


Exercise 9.a - part 2
u(x,t) =  exp(-4*pi**2*t)*sin(2*pi*x)
l =  2
T =  0.5
m =  5
N =  5
alpha =  1
Crank-Nicolson Algorithm :
j ite

### No 9.b

In [42]:
u = lambda x,t: exp(-t)*sin(x)
x = Symbol('x')
t = Symbol('t')

#init variable
#No. 7.b
print("Exercise 9.b")
l = pi.evalf()
T = 0.5
h = pi.evalf()/10
k = 0.05
m = int(l/h)
N = int(T/k)
alpha = 1
x_iter = int(l/h)
xi = np.array([h*i for i in range(1, x_iter)])
#exact calculation of u(x,t)
u_exact = np.array([u(x,t).evalf(subs={x:xi[i], t:T}) for i in range(len(xi))])
#approximation of u(x,t)
w_last = crank_nicolson_parabolic(u, l, T, m, N, alpha)
w_last = w_last[1:-1]

print("\nResult :")
print("x\tu(x,0.5)\tw(i,10)\t|u(x,0.5)-w(i,10)|")
for i in range(len(w_last)):
    print(format(xi[i],'.2f'),"\t",
          format(u_exact[i], '.12f'),"\t",
          format(w_last[i], '.5f'),"\t",
          format(abs(u_exact[i] - w_last[i]), '.5f') )

Exercise 9.b
u(x,t) =  exp(-t)*sin(x)
l =  3.14159265358979
T =  0.5
m =  10
N =  10
alpha =  1
Crank-Nicolson Algorithm :
j iter =  1
x	w[i]
0.314159265358979 	 0.294063576342132
0.628318530717959 	 0.559342160970484
0.942477796076938 	 0.7698684377170509
1.25663706143592 	 0.9050346277910579
1.57079632679490 	 0.951609722749838
1.88495559215388 	 0.9050346277910579
2.19911485751286 	 0.769868437717051
2.51327412287183 	 0.559342160970484
2.82743338823081 	 0.294063576342132

j iter =  2
x	w[i]
0.314159265358979 	 0.27983375835376206
0.628318530717959 	 0.5322754387234175
0.942477796076938 	 0.7326142905697738
1.25663706143592 	 0.8612397512312513
1.57079632679490 	 0.9055610644320236
1.88495559215388 	 0.8612397512312515
2.19911485751286 	 0.7326142905697739
2.51327412287183 	 0.5322754387234175
2.82743338823081 	 0.2798337583537621

j iter =  3
x	w[i]
0.314159265358979 	 0.2662925252030687
0.628318530717959 	 0.5065184826701397
0.942477796076938 	 0.6971628819316718
1.25663706143592